In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

#### SF 2018

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220228/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz025 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.25-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.5-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz150 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.5-20220328/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/"

#### SF 2019

In [3]:
loc_2019_transCap05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_capacity_0.5-20220329/beam/year-2019-iteration-5/ITERS/it.0/"

In [4]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_baseline + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:20: DtypeWarning: Columns (5,7,12,13,14,15,17,23,29,32,44,53,55,56) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 34s


In [5]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
eventsSF['scenario'] = "baseline"
eventsSF['year'] = '2018'

In [7]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [8]:
eventsSFCopy = eventsSF.copy()

In [9]:
# deadhead = eventsSF[eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)
# When there are no riders. Driver and person are not numbers, but include "Agent"

In [10]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [11]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [12]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [13]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [14]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [15]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [16]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

Wall time: 1min 47s


In [17]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [18]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [19]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [20]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3380, 65)

In [21]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

Wall time: 4min 23s


In [22]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [23]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(0, 66)

In [24]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [25]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', nan, 'car', 'car_hov2', 'walk', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bike_transit']


In [26]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', nan, 'car', 'walk', 'car_hov2', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [27]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'TeleportationEvent', 'arrival', 'actstart', 'PersonEntersVehicle', 'PathTraversal', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'PersonLeavesVehicle', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [28]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

Wall time: 18.2 s


In [29]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [30]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [31]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [32]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [33]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [34]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [35]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [36]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [37]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [38]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding census blocks

In [39]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [40]:
BGs = gpd.read_file('C:/Shared-Work/Data/Scenario_2010_shp/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [41]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Wall time: 1h 18s


In [42]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Wall time: 1h


#### Adding new columns

In [43]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [44]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [45]:
eventsSF['duration_total'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [46]:
eventsSF['distance_total'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [47]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [48]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_total'], np.nan)

In [49]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_total'], np.nan)

In [50]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_total'], np.nan)

In [51]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_total'], np.nan)

In [52]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_total'], np.nan)

In [53]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_total'], np.nan)

In [54]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_total'], np.nan)

In [55]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_total'], np.nan)

In [56]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_total'], np.nan)

In [57]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_total'], np.nan)

In [58]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [59]:
#eventsSF = eventsSF.set_index('IDMerged')

In [60]:
eventsSF.modeBEAM_rh.unique()

array(['hov3_teleportation', nan, 'car', 'walk', 'ride_hail', 'car_hov2',
       'walk_transit', 'bus', 'subway', 'hov2_teleportation', 'rail',
       'bike', 'car_hov3', 'tram', 'cable_car', 'drive_transit',
       'ride_hail_pooled', 'ride_hail_transit', 'bike_transit'],
      dtype=object)

In [61]:
eventsSF.primaryFuelType.unique()

array([nan, 'Food', 'Gasoline', 'Diesel', 'Electricity', 'Biodiesel'],
      dtype=object)

In [62]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [63]:
#eventsSF['fuel_marginal_car_primary'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['modeBEAM_rh']=='car')|
                                         #(eventsSF['modeBEAM_rh']=='hov3_teleportation')|(eventsSF['modeBEAM_rh']=='hov2_teleportation')|
                                         #(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')), 
                                #eventsSF['primaryFuel'], np.nan)

In [64]:
#eventsSF['fuel_marginal'] = np.where((eventsSF['modeBEAM_rh']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                #eventsSF['primaryFuel'], np.nan)

In [65]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [66]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [67]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [68]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [69]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [70]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [71]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [72]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [73]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [74]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Previous code for mode choice 

In [75]:
# Make a new column which determined mode choice numbers for each person and trip
#s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [76]:
#eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [77]:
#eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [78]:
# Just showing the mode choice after replanning
#eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

#### Only works if we have one replanning event

In [68]:
# Showing both mode choices before and after the replanning event
#eventsSF['mode_choice_act'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [69]:
# If replanned, then the status is 1
#eventsSF['replanning_status'] = np.where(eventsSF['mode_choice_replanned_BEAM'].notna() , 1, np.nan)

In [70]:
# Group by and fill all the rows related to the replanning event
#eventsSF["replanning_status"] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False)['replanning_status'].transform(lambda x: x.ffill().bfill())

In [71]:
# Put all non-replanning events equal to 0
#eventsSF["replanning_status"] = np.where(eventsSF['replanning_status'].isna(), 0, eventsSF['replanning_status']).astype(int)

In [72]:
#eventsSF['mode_choice_actual_BEAM'] = np.where((eventsSF['replanning_status'] == 1) & (eventsSF['type_number']=='ModeChoice01'), np.nan, eventsSF['mode_choice_act'])

In [73]:
#eventsSF = eventsSF.drop(['mode_choice_act'], axis = 1)

In [74]:
#eventsSF[eventsSF['type'] == "Replanning"].tail(16)

In [76]:
#num_pass.to_csv('C:/Shared-Work/Data/CleanData/num_pass.csv', index = False)

In [77]:
#eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

In [78]:
#eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [79]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [80]:
#eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [81]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

#### Mode Choice planned and actual

In [79]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

Wall time: 1min 25s


In [80]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

Wall time: 1min 25s


In [81]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [82]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [83]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [92]:
#eventsSF[eventsSF['type']=='Replanning'].head()

In [84]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [73]:
#q = eventsSF[eventsSF['IDMerged'] == 6902751]

In [74]:
#q.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/fake2.csv')   

In [85]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

Wall time: 1.65 s


In [86]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [87]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [103]:
#transit_data = eventsSF[(eventsSF['modeBEAM_rh'] == "bus") | (eventsSF['modeBEAM_rh'] == "subway") 
#                        | (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'ride_hail_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'bike_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'cable_car') | (eventsSF['modeBEAM_rh'] == 'rail') 
#                        | (eventsSF['modeBEAM_rh'] == 'tram')]

In [ ]:
#eventsSF.loc['Total',:]= df.sum(axis=0)

In [ ]:
#eventsSF.type[eventsSF.type == 'Replanning'].count()

In [ ]:
#Person_Trip_eventsSF.replanning_status[Person_Trip_eventsSF.replanning_status == 0].count()

In [ ]:
#rh_transit5 = eventsSF[eventsSF['IDMerged']== 3671596]

In [ ]:
#rh_transit5.to_csv('C:/Shared-Work/Data/CleanData/rh_transit5.csv', index = False)

In [69]:
eventsSFCopy = eventsSF.copy()

In [133]:
eventsSF.head(500000).to_csv('C:/Shared-Work/Data/CleanData/modeBEAMCHECK.csv', index = False)

In [103]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'duration_total': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'distance_total': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 'transit_subway': np.sum, 'transit_tram': np.sum, 'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum, 'fuel_marginal': np.sum,
            'BlockGroupStart': 'first',
            'BlockGroupEnd': 'last'}).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

Wall time: 35min 10s


In [104]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [105]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_total'] 

In [106]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [107]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged'] == 2192]

In [108]:
#Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [109]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [110]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [111]:
#Person_Trip_eventsSF.door_to_door_time[Person_Trip_eventsSF.door_to_door_time<0].count()

In [112]:
#negative_door = Person_Trip_eventsSF[(Person_Trip_eventsSF.door_to_door_time<0)]

In [113]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [114]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default=np.nan)

In [115]:
#negative_door.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/negative_door.csv')   

In [116]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [119]:
Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base_core_act.csv', index = False)

In [ ]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['mode_choice_planned_BEAM'] == 'walk_transit'].head()

#### Merging with activity sim persons and housholds files

In [2]:
actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/activitysim/"
actloc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220228/activitysim/"
actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/activitysim/"
actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

households = pd.read_csv(actloc_2018_baseline + 'final_households.csv')
persons = pd.read_csv(actloc_2018_baseline + 'final_persons.csv')
tours = pd.read_csv(actloc_2018_baseline +'final_tours.csv')
plans = pd.read_csv(actloc_2018_baseline +'final_plans.csv')
trips = pd.read_csv(actloc_2018_baseline + 'final_trips.csv')

In [121]:
#persons.earning[persons.earning<0].count()

In [3]:
# Merge households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [6]:
hhpersons.head()

,person_id,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,1140762,115000.0,1,0,0,female,1,2,14,50.0,2,above 60,24.0,-1,1,white,0,63,1406,no,0,0,1005,1,1,3,-122.225696,37.857091,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,4.519702,-1,NaN,0.0,1262,13.822017,0.004654,True,1.0,135.266660,556.6823,1.0,True,N,True,False,False,False,NaN,False,False,0,0,0,20,2,0,1,1,0,0,0,2,1,0.0,1,6001,no,60014001001046,0,no,63,1,own not recent,one,yes,1,one,118500.0,gt35-lt65,2010000113222,1.0,white,gt100-lt150,0,one,0,no,1,2,1005,1,0.097,126428,118.5,4,12.86,4.519702,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
1,1336452,82000.0,1,0,0,female,1,2,16,37.0,5,above 60,22.0,-1,1,white,0,63,401,no,0,0,1005,1,1,3,-122.232831,37.856070,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,7.220078,-1,NaN,0.0,401,13.955740,0.002449,False,3.0,1.091662,775.9439,1.0,False,N,True,False,False,False,NaN,False,False,0,0,0,20,3,0,1,2,0,0,0,3,1,0.0,1,6001,no,60014001001049,0,no,63,1,own not recent,one,yes,1,one,129000.0,gt35-lt65,2013000127260,1.0,white,gt100-lt150,0,one,0,no,1,2,1005,1,0.097,15051,129.0,4,12.86,7.220078,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
2,1956965,190000.0,1,0,0,female,1,2,24,50.0,5,35 to 60,23.0,-1,6,asian,0,57,401,no,0,0,1005,1,1,3,-122.226436,37.855050,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,12.171989,-1,NaN,0.0,401,14.107443,0.002449,False,3.0,1.091662,775.9439,1.0,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,1,6001,no,60014001001054,0,no,57,6,own not recent,one,no,1,one,196000.0,gt35-lt65,2013000674070,1.0,asian,gt150,0,one,0,no,1,2,1005,1,0.097,208708,196.0,4,12.86,12.171989,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
3,5171467,0.0,0,0,0,female,1,2,65,0.0,5,3

In [8]:
hhpersons[hhpersons['serialno']==2009000000022]

,person_id,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
603307,2498343,100000.0,1,0,0,male,1,1,2428308,40.0,3,35 to 60,21.0,-1,1,white,0,58,1393,no,0,0,1375,1,1,3,-122.762097,38.439462,False,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0,True,1375,9.894720,-1,NaN,0.000000,1386,12.550070,4.913054,False,5.0,42.884400,154.874970,1.000000,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,2,6097,no,60971530062004,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1375,4,0.097,198632,150.0,4,12.86,9.894720,1.0,2,2,0,0,0,0,0,1,True,False,False,False,2,1.000000,0,2,2,0,0,2,True,0_tours,0
603308,2498344,30000.0,0,1,0,male,2,1,2428308,20.0,5,20 to 35,20.0,1363,1,white,0,29,-1,no,2,0,1375,3,3,2,-122.762097,38.439462,False,True,True,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,3,False,1375,9.894720,1363,4.495997,37.795480,-1,NaN,NaN,False,NaN,0.000000,0.000000,0.000000,False,M,True,False,False,False,school1,False,False,1,0,0,0,0,0,0,0,0,0,0,0,1,0.0,2,6097,no,60971530062004,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1375,4,0.097,198632,150.0,4,12.86,9.894720,1.0,2,2,0,0,0,0,0,1,True,False,False,False,2,1.000000,0,2,2,0,0,2,True,0_tours,0
606307,1988405,100000.0,1,0,0,male,1,1,2438428,40.0,3,35 to 60,21.0,-1,1,white,0,58,1377,no,0,0,1381,1,1,3,-122.754259,38.375504,False,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0,True,1381,4.323352,-1,NaN,0.000000,1380,12.642690,3.425518,False,4.0,23.366467,98.666855,0.822224,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,2,6097,no,60971533003040,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1381,4,0.097,261004,150.0,4,12.86,4.323352,1.0,2,2,0,0,0,0,0,1,True,False,False,Fal

In [159]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['person_id']).reset_index(drop=True)
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [160]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)
hhperTours = hhperTours.sort_values(by=['person_id','tour_id']).reset_index(drop=True)
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [170]:
households.tail(11)

,household_id,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
274772,2894865,0,1.0,2,6001,no,60014287001008,1,no,51,1,rent not recent,two,yes,2,one,30400.0,gt35-lt65,2009001294011,1.0,white,gt30-lt60,0,one,0,yes,2,7,961,4,0.097,212013,30.40,2,8.81,16.424760,1.0,2,2,0,0,0,0,0,0,True,False,False,False,2,1.000000,0,2,2,0,0,2,True,0_tours,0
274773,2894866,0,0.0,2,6085,yes,60855063014004,1,yes,39,8,rent not recent,four or more,yes,2,two or more,77400.0,gt35-lt65,2011000649587,3.0,other,gt60-lt100,0,two or more,1,no,8,7,471,4,0.097,210028,77.40,3,10.44,21.461305,5.0,4,4,4,2,2,0,0,2,True,False,True,False,2,1.401020,1,6,3,1,3,5,True,0_tours,0
274774,2894872,0,0.0,3,6081,no,60816125003014,1,no,51,1,own not recent,two,yes,1,two or more,126000.0,gt35-lt65,2013000698734,2.0,white,gt100-lt150,0,two or more,0,no,2,3,340,4,0.097,23977,126.00,4,12.86,3.374704,0.0,2,2,0,0,0,0,0,0,True,False,True,False,3,0.893359,0,2,2,0,0,2,True,0_tours,0
274775,2894883,1,0.0,1,6097,no,60971529032004,0,no,58,1,rent not recent,one,no,2,two or more,50000.0,gt35-lt65,2012000154637,0.0,white,gt30-lt60,0,none,0,no,1,6,1372,1,0.097,187300,50.00,2,8.81,3.558924,1.0,1,1,0,0,0,0,0,0,False,True,False,False,1,0.000000,0,1,1,0,0,1,False,0_tours,0
274776,2894892,0,0.0,2,6013,yes,60133200012016,1,no,39,1,own not recent,four or more,yes,1,two or more,175000.0,gt35-lt65,2012000813112,2.0,white,gt150,0,two or more,0,no,4,3,1095,4,0.097,120195,175.00,4,12.86,35.334053,2.0,2,2,2,0,2,0,0,0,True,False,False,False,2,1.797032,0,4,2,0,2,4,True,0_tours,0
274777,2894911,1,1.0,3,6001,yes,60014506041004,1,no,63,6,own not recent,four or more,yes,1,two or more,164100.0,gt35-lt65,2011001020658,1.0,asian,gt150,0,one,0,yes,4,4,741,4,0.097,222129,164.10,4,12.86,5.055393,3.0,3,3,1,0,1,0,0,0,True,False,False,False,3,1.000000,0,4,3,0,1,4,True,0_tours,0
274778,2894948,1,0.0,2,6013,yes,60133032022032,1,no,64,1,own not recent,four or more,yes,1,two or more,404000.0,gt35-lt65,2009000154125,1.0,white,gt150,0,one,0,no,6,4,1178,4,0.097,101991,404.00,4,12.86,3.158078,5.0,5,5,1,0,1,0,2,1,True,False,False,False,2,1.000000,0,6,5,0,1,6,True,0_tours,0
274779,2894956,0,0.0,2,6081,no,60816083002007,1,no,30,1,rent recent,two,no,2,none,57110.0,lt35,2011001280998,2.0,white,gt30-lt60,1,two or more,0,no,2,7,269,4,0.097,238057,57.11,2,8.81,5.431769,0.0,2,2,0,0,0,0,0,2,True,False,False,False,2,2.000000,0,2,2,0,0,2,True,0_tours,0
274780,2894964,0,0.0,2,6085,yes,60855045074001,1,no,33,6,rent not recent,three,no,2,two or more,31300.0,lt35,2009000776741,2.0,asian,gt30-lt60,0,two or more,0,no,3,7,611,4,0.097,204296,31.30,2,8.81,5.298992,1.0,2,2,1,1,0,0,0,2,True,False,False,False,2,2.000000,1,3,2,1,1,2,True,0_tours,0
274781,2894969,0,0.0,2,6085,yes,60855085072009,1,no,41,1,rent not recent,four or more,no,2,two or more,45000.0,gt35-lt65,2010001310391,1.0,white,gt30-lt60,0,one,0,no,4,7,445,4,0.097,271926,45.00,2,8.81,10.246107,3.0,3,3,1,0,1,0,1,0,True,False,True,False,2,0.486690,0,4,3,0,1,4,True,1_Main,1


In [162]:
# Merge person_trip level BEAM with activity sim merged files
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [163]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [164]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [165]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [166]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [133]:
# eventsASim.head(10000).to_csv('C:/Shared-Work/Data/CleanData/Merged_SF_Baseline_2018.csv', index = False)

In [134]:
eventsASim.to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/5_18_22/sf_2018_base.csv', index = False)

In [ ]:
# eventsASim.duration_in_privateCar.sum()

In [136]:
eventsASim.head(30000).to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/5_18_22/sf_2018_base_test.csv', index = False)

In [148]:
persons.head(10)

,person_id,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours
546238,1140762,115000.0,1,0,0,female,1,2,14,50.0,2,above 60,24.0,-1,1,white,0,63,1406,no,0,0,1005,1,1,3,-122.225696,37.857091,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,4.519702,-1,NaN,0.0,1262,13.822017,0.004654,True,1.0,135.266660,556.682300,1.000000,True,N,True,False,False,False,NaN,False,False,0,0,0,20,2,0,1,1,0,0,0,2
169064,1336452,82000.0,1,0,0,female,1,2,16,37.0,5,above 60,22.0,-1,1,white,0,63,401,no,0,0,1005,1,1,3,-122.232831,37.856070,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,7.220078,-1,NaN,0.0,401,13.955740,0.002449,False,3.0,1.091662,775.943900,1.000000,False,N,True,False,False,False,NaN,False,False,0,0,0,20,3,0,1,2,0,0,0,3
707494,1956965,190000.0,1,0,0,female,1,2,24,50.0,5,35 to 60,23.0,-1,6,asian,0,57,401,no,0,0,1005,1,1,3,-122.226436,37.855050,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,12.171989,-1,NaN,0.0,401,14.107443,0.002449,False,3.0,1.091662,775.943900,1.000000,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0
306895,5171467,0.0,0,0,0,female,1,2,65,0.0,5,35 to 60,22.0,-1,1,white,0,42,-1,no,0,0,1005,4,3,3,-122.241097,37.857473,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,False,1005,8.527366,-1,NaN,0.0,-1,NaN,NaN,False,NaN,0.000000,0.000000,0.000000,False,N,True,False,False,False,NaN,False,False,0,0,0,21,3,0,1,1,0,1,0,3
268311,5756287,54000.0,1,0,0,female,1,2,72,30.0,5,35 to 60,18.0,-1,1,white,0,49,401,no,0,0,1005,1,1,3,-122.226110,37.852014,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,48.166088,-1,NaN,0.0,401,13.770918,0.002449,False,3.0,1.091662,775.943900,1.000000,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0
664174,5945850,71000.0,1,0,0,female,1,2,74,40.0,5,35 to 60,21.0,-1,6,asian,0,47,401,no,0,0,1005,1,1,3,-122.244304,37.867695,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,6.807836,-1,NaN,0.0,401,13.926352,0.002449,False,3.0,1.091662,775.943900,1.000000,False,M,True,False,False,False,work1,False,False,1,1,0,1,1,0,0,0,0,1,0,1
549044,6340254,93000.0,1,0,0,male,1,1,79,40.0,3,35 to 60,21.0,-1,1,white,0,52,952,no,0,0,1005,1,1,3,-122.227937,37.854097,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,8.257546,-1,NaN,0.0,955,13.933332,8.911579,False,3.0,87.566666,40.870224,0.340585,False,H,False,False,False,False,NaN,False,False,0,0,0,0,0,0,0,0,0,0,0,0
308657,6829620,65000.0,1,0,0,male,1,1,85,40.0,5,35

In [143]:
#col_list = ["SERIALNO", "DIVISION", "SPORDER", "REGION", "AGEP", "DIS", "SEX"]
#pums = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/psam_p06.csv', usecols=col_list)
pums = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/psam_p06.csv')

In [173]:
pums_short = pums.head(1000)

In [175]:
pums_short.to_csv('C:/Shared-Work/Data/CleanData/pums.csv', index = False)

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

#plt.figure(figsize=(13, 6))
#sns.heatmap(households, vmax=1, annot=True, linewidths=.5)
#plt.xticks(rotation=30, horizontalalignment='right')
#plt.show()

In [ ]:
#import missingno as msno
#%matplotlib inline
#msno.bar(SFbaseline2018)
#msno.heatmap(persons)

In [ ]:
#from io import StringIO # python3; python2: BytesIO 
#import boto
#import boto3
#
#conn = boto.s3.connect_to_region('us-east-2')
#bucket = conn.get_bucket('beam-core-act')
#csv_buffer = StringIO()
#eventsASim.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
#s3_resource.Object(bucket, 'eventsASim.csv').put(Body=csv_buffer.getvalue())